## $R_0$ calculation (Georgia 2nd wave)

In [1]:
import math

import numpy as np
import pandas as pd

from scipy.optimize import curve_fit
from statsmodels.api import OLS
from statsmodels.tools.tools import add_constant

from uncertainties import ufloat 

from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

from bokeh.plotting import figure, output_notebook, show#, output_file
from bokeh.models import ColumnDataSource, HoverTool, WheelZoomTool, PanTool, ResetTool, ColorBar, DatetimeTickFormatter

## Import from the John Hopkins GitHub resource

In [2]:
I_list = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv", sep=',', low_memory=False)
D_list = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv", sep=',', low_memory=False)
R_list = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv", sep=',', low_memory=False)

## From raw table to the structured data

In [3]:
Country = 'Georgia' #'Armenia', 'Azerbaijan'

In [14]:
I_list_country=I_list.drop(columns=[ 'Lat','Long','Province/State'])
I_list_country = I_list_country.set_index('Country/Region')
I_list_country=I_list_country.T
I_list_country[Country]

D_list_country = D_list.drop(columns=[ 'Lat','Long','Province/State'])
D_list_country = D_list_country.set_index('Country/Region')
D_list_country = D_list_country.T
D_list_country[Country]

R_list_country=R_list.drop(columns=[ 'Lat','Long','Province/State'])
R_list_country = R_list_country.set_index('Country/Region')
R_list_country=R_list_country.T
R_list_country[Country]

confirmed = np.array (I_list_country[Country]) #an array of confirmed since 22/1/2020
confirmed = np.append (confirmed, [6192])

recover = np.array (R_list_country[Country]) #an array of recovered since 22/1/2020
recover = np.append (recover, [3120])

death = np.array (D_list_country[Country]) #an array of dead since 22/1/2020
death = np.append (death, [39])

dates = pd.date_range('20200122', periods=len(confirmed))

In [15]:
First_day = 205 #14 August 2020 is approxemately the first day of the second wave
Last_day  = len(dates)

# dates[205]

In [16]:
C_Data = pd.DataFrame({'Date':dates ,'Infectious': confirmed, 'Recover': recover, 'Death':death})

first_infect = C_Data.ne(0).idxmax().Infectious
first_recover = C_Data.ne(0).idxmax().Recover
first_death = C_Data.ne(0).idxmax().Death

C_Data = C_Data.iloc[First_day:Last_day, :]
len(C_Data)

48

## Fitting

\begin{equation}
\begin{split}
    f(x; a, b, c) &=& a e^{bx} + c \qquad\qquad\qquad\qquad\quad – > \mbox{ exp_function}\\
    f(x; a, b, c, d) &=& \frac{a}{1 + \exp({-c (x-d)})} + b \qquad – > \mbox{ logistic_function}
\end{split}
\end{equation}

In [7]:
exp_function = lambda x, a, b, c: a*(math.e**(b*x)+c)
logistic_function = lambda x, a, b, c, d: a / (1 + np.exp(-c * (x - d))) + b

In [8]:
xdata = np.linspace(1, len(C_Data), len(C_Data))
ydata = np.array(C_Data['Infectious'])

In [9]:
exp_params, exp_cov = curve_fit(exp_function, xdata, ydata, p0 = [0, 0, 0])
# log_params, log_cov = curve_fit(logistic_function, xdata, ydata, p0 = [0, 0, 0, 0]) -- logistic curve

In [10]:
# confirmed_pred_log = logistic_function(xdata, al_, bl_, cl_, dl_)
C_Data['Exp_est'] = exp_function(xdata, *exp_params)
R2 = r2_score(ydata,exp_function(xdata, *exp_params))
Lambda = round(exp_params[1],2)
Lambda_sd = math.sqrt(exp_cov[1,1])

## $R_0$ calculation with uncertainties

\begin{equation}
R_0=\frac{(\sigma+\lambda)(\gamma+\lambda)}{\sigma\gamma}
\end{equation}

In [12]:
Gamma = ufloat (1/6, 0.03)
Sigma = ufloat (1/6, 0.03)
Lambda_err = ufloat (Lambda, Lambda_sd)
R0 = (Sigma + Lambda_err) * (Gamma + Lambda_err)/(Sigma*Gamma)
str(R0)

'2.37+/-0.22'

## Bokeh plot

In [13]:
source = ColumnDataSource(C_Data)

hover = HoverTool(names=["dots"],formatters={'@Date': 'datetime'})

TOOLTIPS = [
      ('Date', "@Date{%F}"),
      ('Infectious', "@Infectious"),
      ('Recover', "@Recover"),
      ('Death', "@Death"),
     ]

p = figure(title= "Covid statistics for the 2nd wave in Georgia", tools=[hover, WheelZoomTool(), PanTool(), ResetTool()], tooltips=TOOLTIPS)

p.circle(x = 'Date', y = 'Infectious' , name = "dots", source = source,  size = 10, alpha = 0.8)
p.line(x = 'Date', y = 'Exp_est', source = source, legend_label = "Fitted exponent with λ = "+ str(Lambda) +". R0 = " + str(R0), line_width = 2.5, alpha = 0.8, color = 'crimson')

p.legend.location = "top_left"
p.legend.label_text_font_size = "13pt"
p.sizing_mode = "scale_both"

p.yaxis.major_label_text_font_size = "14px"
p.xaxis.major_label_text_font_size = "14px"

p.title.text_font_size = '16px'

p.xaxis.formatter=DatetimeTickFormatter(
        days=["%d %B %Y"],
        months=["%d %B %Y"]
    )

p.xaxis.major_label_orientation = math.pi/6

p.xaxis.axis_label_text_font_size = '16px'
p.yaxis.axis_label_text_font_size = '16px'

p.yaxis.axis_label = "Number of confirmed cases"

output_notebook()

show(p)

Loading BokehJS ...